In [1]:
import torch
import os
import matplotlib.pyplot as plt
os.chdir("..")

from epilearn.models.SpatialTemporal.STGCN import STGCN
from epilearn.models.SpatialTemporal.MepoGNN import MepoGNN
from epilearn.models.SpatialTemporal.GraphLSTM import GraphLSTM
from epilearn.models.SpatialTemporal.EpiGNN import EpiGNN
from epilearn.models.SpatialTemporal.DASTGN import DASTGN
from epilearn.models.SpatialTemporal.ColaGNN import ColaGNN
from epilearn.models.SpatialTemporal.EpiColaGNN import EpiColaGNN
from epilearn.models.SpatialTemporal.CNNRNN_Res import CNNRNN_Res
from epilearn.models.SpatialTemporal.ATMGNN import ATMGNN

from epilearn.data import UniversalDataset
from epilearn.utils import utils, transforms
from epilearn.tasks.forecast import Forecast

### Configs

In [2]:
# initial settings
device = torch.device('cpu')
torch.manual_seed(7)

lookback = 12 # inputs size
horizon = 3 # predicts size

# permutation is True when using STGCN
permute = True

epochs = 50 # training epochs
batch_size = 50 # training batch size

### Initialize dataset

In [3]:
# load toy dataset
dataset = UniversalDataset()
dataset.load_toy_dataset()

### Initialize model and task

In [4]:
task = Forecast(prototype=CNNRNN_Res, dataset=None, lookback=lookback, horizon=horizon, device='cpu')

### Add transforms to dataset

In [ ]:
# transforms = transforms.Compose({"features":[], 'graph': [], 'states': []})

### Train Model

In [5]:
config = None
# for epicolagnn, loss='epi_cola', else loss='mse
# for STGCN, permute_dataset=True
model = task.train_model(dataset=dataset, config=config, loss='mse', epochs=5, permute_dataset=False) # instead of config, we can also dircetly input some parameters

 80%|████████  | 4/5 [00:00<00:00, 16.35it/s]

######### epoch:0
Training loss: 0.7122649031300698
Validation loss: 0.6276538968086243


100%|██████████| 5/5 [00:00<00:00, 16.77it/s]



Final Training loss: 0.4646987828516191
Final Validation loss: 0.4924061596393585
Test MAE: 224.5708770751953


### Evaluate model

In [ ]:
# evaluation = task.evaluate_model(dataset, config=config)